**Import Modules**

In [1]:
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import ast
import csv

2023-12-13 17:36:23.770540: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 17:36:23.772702: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 17:36:23.819662: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-13 17:36:23.820743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 17:36:24.595514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

**Load Model**

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


**Load Dataset**

In [5]:
with open('dataset/recycling_article_dataset.csv', 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter = ',')
  next(csvfile)
  judul_artikel = [row[0].replace("\n", "") for row in reader]

print(len(judul_artikel))
print(judul_artikel)

159
['Membuat Keset dari Kain Perca dengan Menyulam', 'Tote Bag dari Kain Perca', 'Sarung Kabel dari Kain Perca', 'Bandana dari Kain Perca', 'Membuat Keset dari Kain Perca dengan Kawat Kasa', 'Syal Dari Kaos Bekas', 'Tote Bag Kaos Bekas', 'Gelang Kaos Bekas', 'Kaos Ombre', 'Kantung Koin Dari Kaos Bekas', 'Tas Belanja Kaos Bekas', 'Boneka Kaos Bekas', 'Selimut Kaos Bekas', 'Masker dari kain perca', 'Tas Jinjing dari Kain Perca', 'Tas Rias Make Up', 'Selimut dari kain perca', 'Sarung bantal', 'Sarung kabel', 'Taplak Meja dari Kain Perca', 'Membuat Benang dari Kaos', 'Bandana kain perca', 'Membuat Bros dari Kain Perca', 'Gantungan Kunci dari Kain Perca', 'Gorden Dari Kain Perca Yang Sederhana', 'Membuat Wadah Unik dari Kaos Bekas', 'Membuat kertas daur ulang', 'Vas Bunga Dari Koran Bekas', 'Jam Dinding dari Koran', 'Membuat Dompet Dari Anyaman Kertas', 'Keranjang dari Kertas Koran Bekas', 'Membuat Mangkok dari Koran Bekas', 'Kerajinan dari Koran Jadi Kertas Daur Ulang', 'Kerajinan dari Ko

**Preprocess Title (Remove Stopwords)**

In [6]:
stopword_file_path = "dataset/stopwords-id.txt"

with open(stopword_file_path, 'r') as file:
    stopwords = [line.strip() for line in file]

print(len(stopwords))

757


In [7]:
judul_artikel_filtered = []

for title in judul_artikel:
    words = title.split()
    filtered_words = [word for word in words if word.lower() not in stopwords]
    filtered_title = ' '.join(filtered_words)
    judul_artikel_filtered.append(filtered_title)

print(judul_artikel_filtered)

['Keset Kain Perca Menyulam', 'Tote Bag Kain Perca', 'Sarung Kabel Kain Perca', 'Bandana Kain Perca', 'Keset Kain Perca Kawat Kasa', 'Syal Kaos Bekas', 'Tote Bag Kaos Bekas', 'Gelang Kaos Bekas', 'Kaos Ombre', 'Kantung Koin Kaos Bekas', 'Tas Belanja Kaos Bekas', 'Boneka Kaos Bekas', 'Selimut Kaos Bekas', 'Masker kain perca', 'Tas Jinjing Kain Perca', 'Tas Rias Make Up', 'Selimut kain perca', 'Sarung bantal', 'Sarung kabel', 'Taplak Meja Kain Perca', 'Benang Kaos', 'Bandana kain perca', 'Bros Kain Perca', 'Gantungan Kunci Kain Perca', 'Gorden Kain Perca Sederhana', 'Wadah Unik Kaos Bekas', 'kertas daur ulang', 'Vas Bunga Koran Bekas', 'Jam Dinding Koran', 'Dompet Anyaman Kertas', 'Keranjang Kertas Koran Bekas', 'Mangkok Koran Bekas', 'Kerajinan Koran Kertas Daur Ulang', 'Kerajinan Koran Tempat Pensil', 'Kerajinan Koran Bekas Tong Sampah', 'Jam Dinding Koran Bekas', 'Kerajinan Bunga Kertas Koran', 'Tempat Pensil Koran', 'Tempat Tisu Koran Bekas', 'Tempat Tisu Sederhana Koran Bekas', 'Ala

**Translate Title to English**

Title need to be translated to English before encoding because universal-sentence-encoder model can only work with English sentences.

In [9]:
from googletrans import Translator

def translate_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='id', dest='en')
    return translation.text

judul_eng = []

for judul in judul_artikel_filtered:
    translate = translate_to_english(judul)
    judul_eng.append(translate)

print(len(judul_eng))
print(judul_eng)

159
['Patchwork', 'Patchwork tote bag', 'Patchwork cable sarong', 'Patchwork Bandana', 'Patchwork', 'Used t -shirt scarves', 'Used tote bag used shirt', 'Used shirt bracelet', 'Ombre shirt', 'Used t -shirt coin bag', 'Used shirt shopping bag', 'Used shirt dolls', 'Used T -shirt blankets', 'Patchwork mask', 'Patchwork tote bag', 'Makeup makeup bag', 'Patchwork blanket', 'Pillow case', 'Cable sarong', 'Patchwork tablecloth', 'T -shirt thread', 'Patchwork Bandana', 'Patchwork brooch', 'Patchwork key chain', 'Simple patchwork curtains', 'Unique t -shirt container', 'recycled paper', 'Used newspaper flower vase', 'Newspaper wall clock', 'Paper woven paper', 'Used newspaper basket', 'Used newspaper bowl', 'Recycled paper newspaper craft', 'Newspaper crafts', 'Trash canal crafts', 'Used newspaper wall clock', 'Newspaper flower', 'Newspaper pencil', 'Used newspaper tissue', 'Simple newspaper tissue place', 'The base of the newspaper cup', 'A bouquet of newspaper flowers', 'Used newspaper flowe

**Correct Translation**

In [10]:
judul_eng[121] = "Tree Shaped Home Decoration"
judul_eng[114] = "Wall Flower Vase from Glass Jar"
judul_eng[118] = "Ornamental Plants of Mother-in-Law's Tongue from Crackle Plastic"
judul_eng[122] = "Piggy Bank from a Plastic Bottle"

**Encode Title**

In [13]:
judul_embeddings = embed(judul_eng)
judul_embeddings = np.array(judul_embeddings)

index = 0

print(judul_eng[index])
print(len(judul_embeddings[index]))
print(judul_embeddings[index])

Patchwork
512
[-3.97124365e-02 -4.31467667e-02 -7.99282733e-03  3.00785862e-02
  3.00580803e-02 -5.19009191e-04 -6.49927035e-02 -5.24889790e-02
  1.78139228e-02  4.66566719e-02  3.88202746e-03  5.46557903e-02
  6.08177520e-02 -2.69893035e-02  4.85278033e-02  7.49173760e-02
  6.47204667e-02  9.84328426e-03 -6.60191814e-04 -2.57106032e-02
  1.68264657e-02  6.91706613e-02  3.36165912e-02 -6.04919791e-02
  3.46444733e-02  1.25996368e-02 -1.52427936e-02  3.60912308e-02
  4.61333105e-03  2.61290949e-02  9.29184631e-02 -1.56816430e-02
 -7.92990476e-02 -4.20086086e-02  6.74966350e-02  7.34092817e-02
  2.14893762e-02 -5.38668130e-03  3.29673961e-02 -3.86439525e-02
 -1.15269488e-02 -4.40859757e-02 -5.11377342e-02 -6.56277537e-02
 -6.78485259e-02  3.78265604e-02 -2.57464796e-02  1.93280037e-02
  5.05662598e-02 -3.12331133e-02  7.54499435e-02 -3.59507464e-02
  6.54379129e-02  4.43768641e-03  7.16282055e-02 -4.79663312e-02
 -9.51930881e-03 -7.44147971e-02  7.44526461e-02  1.48138134e-02
 -4.8748210

**Write a CSV File Containing Title and the Embedding**

In [14]:
data = list(zip(judul_artikel, judul_eng, judul_embeddings))

csv_file_path = 'dataset/recycling_article_embedding_clean.csv'

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    writer.writerow(['judul artikel', 'judul artikel eng', 'embedding'])
    
    # Write data
    writer.writerows(data)

print(f"CSV file '{csv_file_path}' has been created.")

CSV file 'dataset/recycling_article_embedding_clean.csv' has been created.


**Compute Similiarity**

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(judul_embeddings, judul_embeddings)

# Loop through each sentence and find the 5 most similar sentences (excluding itself)
top_similar_sentences = []

for i in range(len(judul_embeddings)):
    # Sort the similarity scores in descending order and get the indices of the top 5 (excluding the current sentence)
    top_indices = np.argsort(similarity_matrix[i])[::-1][:6]  # Including the current sentence itself
    top_indices = [index for index in top_indices if index != i][:5]  # Exclude the current sentence

    # Create a list of tuples containing (sentence_index, similarity_score)
    similar_articles = [(index, similarity_matrix[i][index]) for index in top_indices]

    # Append the list of similar articles to the final result
    top_similar_sentences.append(similar_articles)

# Print or use the top_similar_articles list as needed
print(top_similar_sentences)

[[(4, 0.9999998), (22, 0.7817277), (19, 0.7568165), (16, 0.7421214), (3, 0.66862637)], [(14, 1.0), (16, 0.68890816), (19, 0.6683071), (21, 0.66568273), (3, 0.66568273)], [(18, 0.72599804), (19, 0.6359228), (22, 0.5815238), (16, 0.5550788), (24, 0.5453267)], [(21, 1.0000001), (22, 0.712329), (19, 0.6770296), (13, 0.67696774), (0, 0.66862637)], [(0, 0.9999998), (22, 0.7817277), (19, 0.7568165), (16, 0.7421214), (3, 0.66862637)], [(12, 0.727385), (6, 0.61070764), (7, 0.59997463), (10, 0.59029424), (9, 0.579768)], [(10, 0.78568554), (9, 0.7574779), (7, 0.65871733), (1, 0.65394664), (14, 0.65394664)], [(6, 0.65871733), (11, 0.64011824), (10, 0.60796136), (5, 0.59997463), (9, 0.5764471)], [(21, 0.51203454), (3, 0.51203454), (10, 0.47319186), (5, 0.47191957), (11, 0.46631157)], [(6, 0.7574779), (10, 0.7178974), (5, 0.579768), (7, 0.5764471), (12, 0.5726242)], [(6, 0.78568554), (9, 0.7178974), (7, 0.60796136), (5, 0.59029424), (11, 0.58286417)], [(7, 0.64011824), (12, 0.5960684), (10, 0.582864

In [19]:
for index, top_similiarity in enumerate(top_similar_sentences):
    print(judul_artikel[index])

    print("[", end='')
    for index, score in top_similiarity:
        print(f"{judul_artikel[index]}, ", end='')

    print("]")
    print()

Membuat Keset dari Kain Perca dengan Menyulam
[Membuat Keset dari Kain Perca dengan Kawat Kasa, Membuat Bros dari Kain Perca, Tote Bag dari Kain Perca, Tas Jinjing dari Kain Perca, Taplak Meja dari Kain Perca, ]

Tote Bag dari Kain Perca
[Tas Jinjing dari Kain Perca, Tote Bag Kaos Bekas, Selimut dari kain perca, Taplak Meja dari Kain Perca, Bandana dari Kain Perca, ]

Sarung Kabel dari Kain Perca
[Sarung kabel, Taplak Meja dari Kain Perca, Gantungan Kunci dari Kain Perca, Gorden Dari Kain Perca Yang Sederhana, Membuat Bros dari Kain Perca, ]

Bandana dari Kain Perca
[Bandana kain perca, Taplak Meja dari Kain Perca, Masker dari kain perca, Membuat Bros dari Kain Perca, Selimut dari kain perca, ]

Membuat Keset dari Kain Perca dengan Kawat Kasa
[Membuat Keset dari Kain Perca dengan Menyulam, Membuat Bros dari Kain Perca, Sarung Kabel dari Kain Perca, Taplak Meja dari Kain Perca, Selimut dari kain perca, ]

Syal Dari Kaos Bekas
[Kantung Koin Dari Kaos Bekas, Selimut Kaos Bekas, Tas Belanj